In [1]:
import psycopg2
import bcrypt

In [2]:
d = {
    "host": "localhost",
    "database": "dbex2",
    "user": "postgres",
    "password": "12345678"
}
conn = psycopg2.connect(**d)

In [4]:
def login(conn, email, password):
    cur = conn.cursor()
    cur.callproc("get_password", [email])
    user = cur.fetchone()
    if user is None:
        return None
    else:
        user = {
            "id": user[0],
            "name": user[1],
            "email": user[2],
            "phone": user[3],
            "street": user[4],
            "password": user[5],
            "zip": user[6]
        }
    if bcrypt.checkpw(bytes(password, "utf-8"), bytes(user["password"], "utf-8")):
        return user
    else:
        return None

def delete_user(conn, email):
    cur = conn.cursor()
    cur.execute("CALL delete_user('{}')".format(email))

def create_user(conn, user):
    u = user
    u["password"] = bcrypt.hashpw(bytes(u["password"], "utf-8"), bcrypt.gensalt(4)).decode("utf-8")
    u = (u["name"], u["email"], u["phone"], u["street"], u["password"], int(u["zip"]))
    cur = conn.cursor()
    
    try:
        cur.callproc("create_user", u)
        res = cur.fetchone()
        return res
    except:
        cur.execute("ROLLBACK")

def get_user_info(conn, email):
    cur = conn.cursor()
    cur.callproc("get_user_info", [email])
    res = cur.fetchone()
    
    if res is None:
        return None
    user = {
        "id": res[0],
        "name": res[1],
        "email": res[2],
        "phone": res[3],
        "street": res[4],
        "password": res[5],
        "lat": res[6],
        "lon": res[7],
        "zip": res[8],
        "region": res[9]
    }
    return user

In [15]:
user = {
    "name": "daniel",
    "email": "da@ni.el",
    "phone": "12345678",
    "street": "vej24",
    "password": "1234",
    "zip": "2680"
}
d = delete_user(conn, user["email"])
c = create_user(conn, user)
l = login(conn, "da@ni.el", "1234")
i = get_user_info(conn, "da@ni.el")



for k,v in l.items():
    print(k,"     \t",v)

    
print("\n------------------------------------\n")

for k,v in i.items():
    print(k,"     \t",v)

id      	 41843
name      	 daniel
email      	 da@ni.el
phone      	 12345678
street      	 vej24
password      	 $2b$04$bb7.sGKRQE1.pBST3IWvbuSZMw1WNSidNxOA/R4YdJZ7BsGSoZP3y
zip      	 2680

------------------------------------

id      	 41843
name      	 daniel
email      	 da@ni.el
phone      	 12345678
street      	 vej24
password      	 $2b$04$bb7.sGKRQE1.pBST3IWvbuSZMw1WNSidNxOA/R4YdJZ7BsGSoZP3y
lat      	 12.26072581
lon      	 55.5150734
zip      	 2680
region      	 Solrød Strand
